In [27]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

In [28]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

Zur Durchführung der beschriebenen Tests werden zunächst globale Variablen definiert. Diese lauten wir folgt:
* `STOCKFISH_PATH`: Ein String, der den Pfad zur bereits heruntergeladenen Stockfish-Engine bereitstellt. Sofern dies noch nicht geschehen ist, kann das unter diesem [Link](https://stockfishchess.org/) geschehen.
* `VERBOSE`: Ein Boolean, das die zusätzlichen Konsolenausgaben zum Debuggen anzeigt.
* `STOCKFISH`: Ein Objekt, das für die Verwendung der Stockfish-Engine benötigt wird.

In [29]:
STOCKFISH_PATH = "./stockfish/stockfish.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

Weiter werden drei globale Variablen für die bereits berechneten $S_n$ Mengen definiert.
Diese Mengen sind in Form einer Datei innerhalb des `S_n_Results`-Ordner zu finden.
Sie werden mit der Funktion `load_data(filename)` aus dem Notebook `TODO` ausgelesen.
Eine detaillierte Beschreibung der Funktion ist dort zu finden.
Bei diesen Variablen handelt es sich um:
* `S_N_FILE`: Ein String, der den Dateinamen ohne die `.chessAI` Endung beinhatet.
* `PIECE_LIST`: Eine Liste von `chess.Piece`, die bei der Berechnung der $S_n$ Mengen verwendet wurde.
* `S_N_INTEGERS`: Eine Liste von Mengen. Diese Mengen enthalten die Integer-Repräsentationen der einzigartigen Spielsituationen der $S_n$ Menge.

In [30]:
# copied from `03_play_against_ai_restructure.ipynb
# Maybe outsource this function in separate file
def load_data(filename):
    s_n_sequence_integers = []
    with ZipFile("S_n_Results/" + filename + ".chessAI") as zipped:
        with zipped.open(filename + ".pickle") as calculation:
            tmp = pickle.loads(calculation.read())
            piece_list = tmp[0]
            for item in tmp[1:]:
                s_n_sequence_integers.append(
                    item)
    return piece_list, s_n_sequence_integers

SWAPS = {
    "vertical": {x: x ^ 56 for x in range(64)},
    "horizontal": {x: x ^ 7 for x in range(64)},
    "rotate_right": {x: (((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
    "rotate_180": {x: x ^ 63 for x in range(64)},
    "rotate_left": {x: (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)},
    "diagonal": {x: ((x >> 3) | (x << 3)) & 63 for x in range(64)},
    "anti_diagonal": {x: (((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)}
}

# Already outsourced in 11_mirroring.ipynb
# Mirroring mit 7 Bit Darstellung
def mirror_board(board_int, mirror: dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    n = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1

    for i in range(n):
        result |= mirror[board_int & 127] << 7 * i + 1
        board_int = board_int >> 7
    return result


def mirror_all_directions(board_int):
    result = set()
    for name, swap in SWAPS.items():
        result.add(mirror_board(board_int, swap))
    return result

# Also outsource with maybe load_data(filename) ?
def gen_all_integers(s_n_integers):
    result = []
    for sequence in s_n_integers:
        int_set = set()
        for int_board in sequence:
            int_set.add(int_board)
            int_set |= mirror_all_directions(int_board)
        result.append(int_set)
    return result

In [31]:
# Copied from `03_play_against_ai`
def get_color(turn):
    if turn:
        return "White"
    else:
        return "Black"

In [32]:
S_N_FILE = "s_n_queen_24_05"

PIECE_LIST, S_N_INTEGERS = load_data(S_N_FILE)
S_N_INTEGERS = gen_all_integers(S_N_INTEGERS)

## Vorbereitung der Tests

Der Vergleich findet zwischen der KI und der Stockfish-Engine statt.
Hierzu müssen einerseits Funktionen definiert werden, die ein Spiel Stockfish (Schwarz) vs. KI (Weiß) simulieren.
Andererseits gilt es auch Funktionen zu definieren, die ein Spiel Stockfish (Schwarz) vs. Stockfish (Weiß) simulieren.

Mit der Funktion `find_next_move(fen, s_index)` kann nun für eine Partie Stockfish (Schwarz) vs. KI (Weiß) der nächste Zug bestimmt werden.
Hierzu werden zunächst zwei Hilfsfunktionen (`ki_move(cur_board, s_index)`, `stockfish_move(cur_board, s_index)`) definiert.

Die Funktion `ki_move(cur_board, s_index)` bestimmt für eine vorgegebene Spielsituation einen Spielzug für die KI.
Dieser Zug wird mithilfe der bereits berechneten $S_n$-Mengen bestimmt. Dabei wird für die Spielsituation jeder mögliche legale Zug geschaut, ob der Folgezug in einer geringen $S_n$-Menge landet. Die Funktion benötigt hierzu die Parameter:
* `cur_board`: Ein `chess.Board`-Objekt, bei dem gerade Weiß am Zug ist.
* `s_index`: Ein Integer, der das $n$ der $S_n$-Menge widerspiegelt, in dem sich `cur_board` befindet.

Als Rückgabe werden zwei Werte zurückgegeben.
Der erste Wert ist entweder der übergebene `s_index`-1 oder der Wert -1.
Handelt es sich um `s_index`-1, wurde ein passender Zug für die KI gefunden.
In diesem Fall wird neben dem neuen Index noch der `chess.Move` zurückgegeben.
Sofern der erste Wert -1 beträgt, konnte kein passender `chess.Move` für die KI gefunden werden und es wird `None` zusätzlich zurückgegeben.

In [33]:
def ki_move(cur_board, s_index):
    for move in cur_board.legal_moves:
        cur_board.push(move)
        cur_int = to_integer(cur_board, PIECE_LIST)
        _tmp = find_situation_in_sequence(cur_int , [S_N_INTEGERS[s_index - 1]])
        # @Lukas brauchen wir hier wirklich eine separate Variable?
        s_index_tmp = s_index - 1
        if _tmp != -1:
            cur_board.pop()
            return s_index_tmp, move
        cur_board.pop()
    return -1, None

Das Gegenstück zu `ki_move(cur_board, s_index)` ist die Funktion `stockfish_move(cur_board, s_index)`.
Diese bestimmt für Stockfish den nächsten Zug. Hierzu braucht sie ebenfalls zwei Parameter:
* `cur_board`: Ein `chess.Board`-Objekt, bei dem gerade Schwarz am Zug ist.
* `s_index`: Ein Integer, der das $n$ der $S_n$-Menge widerspiegelt, in dem sich `cur_board` befindet.

Die Rückgabe baut sich gleichermaßen aus dem neuen `s_index` und dem ausgeführten `chess.Move` auf. Dabei kann der neue `s_index` auch < `s_index`-1 sein.

In [34]:
def stockfish_move(cur_board, s_index):
    move = chess.Move.from_uci(STOCKFISH.get_best_move())
    cur_board.push(move)
    cur_int = to_integer(cur_board, PIECE_LIST)
    s_index =  find_situation_in_sequence(cur_int, S_N_INTEGERS[:s_index])
    cur_board.pop()
    return s_index, move

In [35]:
def find_next_move(fen, s_index):
    global STOCKFISH

    cur_board = chess.Board(fen)

    if VERBOSE:
        print("---"+ get_color(cur_board.turn) +":---")
        print("Starting in S" + str(s_index))

    if cur_board.turn:
        s_index, move = ki_move(cur_board, s_index)
    else:
        STOCKFISH.set_fen_position(fen)
        s_index, move = stockfish_move(cur_board, s_index)

    if VERBOSE and move is not None:
        print("    Move: " + str(move))
        print("    S" + str(s_index))
        print("Ended in S" + str(s_index))

    return s_index, move

> TODO: Beschreibung der Funktion

In [36]:
def all_moves_stock_ai(board):
    moves = []
    cur_int = to_integer(board, PIECE_LIST)
    s_index = find_situation_in_sequence(cur_int, S_N_INTEGERS)

    while s_index > 0:
        cur_fen = board.fen()
        s_index, next_move = find_next_move(cur_fen, s_index)
        board.push(next_move)
        moves.append(next_move.uci())

    if s_index == -1:
        return None

    return moves

> TODO: Beschreibung der Funktion
> TODO: Wird keine FEN mehr notwendig sein, lieber integer und diese in ein Board umwandeln

In [37]:
def stockfish_move_list(board):
    moves = []

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move.uci())

    return moves

# Vergleich Stockfish vs KI
> TODO: Beschreibung der Funktion
> TODO: Überlegung, ob die nächsten drei (in dem Sinne statistischen) Funktionen wirklich notwendig sind; Hintergedanke, wenn Stockfish besser ist, ist die Frage nach dem warum
Kann man als potenzielle Übersicht hinstellen, wobei das Betrachten im Fokus stehen sollte

## Statistiken durch den Vergleich

In [38]:
def compare_move_lists(move_count_list):
    equal = 0
    ki_better = 0
    stockfish_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff == 0:
            equal += 1
        elif diff < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    return equal, ki_better, stockfish_better

> TODO: Beschreibung der Funktion


In [39]:
def get_average_difference(move_count_list):
    percentual_ki = []
    percentual_stock = []
    avg_ki_better = 0
    avg_stock_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff < 0:
            percentual_ki.append(round(1 - (ki_move / stock_move), 4))
        elif diff > 0:
            percentual_stock.append(round(1 - (stock_move / ki_move), 4))
    if len(percentual_ki) != 0:
        avg_ki_better = sum(percentual_ki) / len(percentual_ki)
    if len(percentual_stock) != 0:
        avg_stock_better = sum(percentual_stock) / len(percentual_stock)
    return avg_ki_better, avg_stock_better

> TODO: Beschreibung der Funktion
> TODO: Wenn Statistiken nicht mehr benötigt werden, müssen Ergebnisse auch nicht mehr in Datei geschrieben werden

In [40]:
def write_result_to_file(filename, sequence_index, move_count_list):
    equal, ki_better, stockfish_better = compare_move_lists(move_count_list)
    avg_ki_better, avg_stock_better = get_average_difference(move_count_list)
    count = ki_better + stockfish_better + equal
    f = open("Tests/" + filename + ".txt", "a+")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better / count) * 100, 2)) + "% besser.\n")
    f.write("Die KI war zu " + str(round((ki_better / count) * 100, 2)) + "% besser.\n")
    f.write("Stockfish und die KI haben zu " + str(
        round((equal / count) * 100, 2)) + "% die gleichen Ergebnisse erzielt.\n")
    f.write("Sofern die KI besser war, hat sie durchschnittlich " + str(
        round(avg_ki_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.write("Sofern Stockfish besser war, hat sie durchschnittlich " + str(
        round(avg_stock_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.close()

## Anzeigen in Stroetmann-Schreibweise der Spielverläufe Stockfish und KI.

Die Funktion `display_both_move_lists(moves_ai, moves_stockfish)` wird in dem Fall benötigt, wenn Stockfish insgesamt weniger Züge benötigt hat als die selbstgeschriebene KI.
Sie ermöglicht es den Spielverlauf anhand einer Move-Historie miteinander zu vergleichen.
Hierzu werden beide Historien nebeneinander in der Konsole ausgegeben.
Die Parameter der Funktion sind Folgende:
* `moves_ai`: Eine Liste an Strings. Diese enthalten Züge im UCI (Universal Chess Interface) Format. Die Liste wurde bei der Durchführung eines aufgeführten Tests für das Spiel KI vs. Stockfish erstellt.
* `move_stockfish`: Eine Liste an Strings. Diese enthalten Züge im UCI (Universal Chess Interface) Format. Die Liste wurde bei der Durchführung eines aufgeführten Tests für das Spiel Stockfish vs. Stockfish erstellt.

Da die Funktion nur aufgerufen wird, wenn Stockfish vs. Stockfish weniger Züge benötigt, ist kein Abgleich der Listen notwendig.

> TODO: Überlegung, ob man dieses print auch in Form einer Datei abspeichern soll?
> TODO: Kommentare an Funktion anpassen!

In [41]:
# Function only called if moves_stockfish is shorter
def write_both_move_lists(fen, filename, moves_stockfish, moves_ai):
    f = open("Tests/" + filename + ".txt", "a+")
    f.write(fen + "\n")
    len_ai = len(moves_ai)
    len_stock = len(moves_stockfish)
    f.write("AI:" + "\t" + "\t" + "\t" + "\t" + " Stockfish:\n")
    for i in range(0, len_stock, 2):
        if i + 1 < len_ai:
            f.write(str(int((i+2)/2)) + ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] + "\t" + " " + str(int((i+2)/2)) + ". " + moves_ai[i] + " " + moves_ai[i+1] + "\n")
        elif i < len_ai:
            f.write(str(int((i+2)/2)) + ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] + "\t" + " " + str(int((i+2)/2)) + ". " + moves_ai[i] + "     " + "\n" )
        else:
            if i + 1 < len_stock:
                f.write(str(int((i+2)/2))+ ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] + "\n")
            elif i < len_stock:
                f.write(str(int((i+2)/2)) + ". " + moves_stockfish[i] + "\n")
    f.close()

In [42]:
write_both_move_lists("asdfghj", "asdgh2", ["asdf", "asdf", "asdf" , "asdf" , "asdf"], ["asdf", "asdf", "asdf"])

# TESTS

> TODO: Beschreibung der Funktion
> TODO: Sollen alle Testergebnisse auch wirklich in einer Datei abgespeichert werden?
> TODO: Bei allen Tests eine Funktion, die den Vergleich von Stockfish und KI durchführt (ohne prints, nur Berechnung / Aufruf der bereits definierten Funktionen
> Wichtige Abklärung: Wenn sich herausstellt, dass Stockfish besser ist, wann soll die Anzeige passieren / Wie soll die Anzeige passieren? Sollen wir Konsole fluten mit einem / zwei Spielen
> Weitere Überlegung zum Vergleich: Die Move_History in dem Schachformat abspeichern und demnach abgleichen
> Bei Abgleich schauen, ob es an einem "falschen" Zug von weiß (also gewinnender Seite) oder scharz liegt, weil wenn schwarz, dann liegt es an Stockfish und nicht der KI)

In [43]:
def ki_vs_stockfish(board, filename):
    ai_moves = all_moves_stock_ai(board)
    stockfish_moves = stockfish_move_list(board)
    move_count = len(ai_moves)
    cmp_move_count = len(stockfish_moves)
    if move_count > cmp_move_count:
        write_both_move_lists(board.fen(), filename, ai_moves, stockfish_moves)
    move_count_t = tuple((move_count, cmp_move_count, move_count - cmp_move_count))
    return move_count_t

> TODO: Beschreibung der Funktion


In [44]:
def compare_fen_stockfish(fen):
    moves = calculate_all_moves(fen)
    stockfish_moves = stockfish_move_list(fen)

    if Moves is not None:
        print("AI needed " + str(len(moves)) + " moves to beat Stockfish as Black.")
    else:
        print("AI found no way to beat Black.")

    print("Stockfish needed " + str(len(stockfish_moves)) + " moves to win against itself.")

> TODO: Beschreibung der Funktion
> TODO: Bzgl. des Random Boards nochmal überlegen wie sinnig das ist ein Board aus den bisher klassifizierten auszuwählen

In [45]:
import random

def test_random_boards(count):

    filename = "Random_" + str(count) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                           "_") + ".txt"
    count_s_n = len(S_N_INTEGERS)
    move_count_list = []
    for board_c in range(count):
        rand_s_n = S_N_INTEGERS[random.randint(0, count_s_n - 1)]
        rand_board_int = random.choice(rand_s_n)
        rand_board = to_board(rand_board_int, PIECE_LIST)
        move_count_tuple = ki_vs_stockfish(rand_board, filename)
        move_count_list.append(move_count_tuple)
        clear_output()
        print("Analyzed " + str(board_c + 1) + "/" + str(count))

    write_result_to_file(filename, "random", move_count_list)

In [46]:
# test_random_boards(100, S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion


In [47]:
def compare_sequence_stockfish(sequence_index, g_filename=None):
    s_n = S_N_INTEGERS[sequence_index]
    move_count_list = []
    length = len(s_n)
    r = 0
    if g_filename is None:
        filename = "S_" + str(sequence_index) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                                   "_") + ".txt"
    else:
        filename = g_filename

    for board_int in s_n:
        board = to_board(board_int, PIECE_LIST)
        move_count_tuple = ki_vs_stockfish(board, filename)
        move_count_list.append(move_count_tuple)
        clear_output()
        print("Comparing S_" + str(sequence_index) + ":")
        print("Compared " + str(r + 1) + "/" + str(length))
        r += 1
    write_result_to_file(filename, sequence_index, move_count_list)

In [49]:
compare_sequence_stockfish(4)

3004
Comparing S_4:
Compared 112/3004


TypeError: object of type 'NoneType' has no len()

> TODO: Beschreibung der Funktion


In [ ]:
def compare_all_sequences(s_n_sequence_fen, s_n_sequence_short):
    filename = "All_S_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":", "_") + ".txt"
    for i in range(len(s_n_sequence_fen)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i, s_n_sequence_fen, s_n_sequence_short, filename)

In [ ]:
compare_all_sequences(S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion
> TODO: Funktion nutzen, um Moves, wenn Stockfish besser ist anzuzeigen?
> TODO: Möglicherweise auch für einen Vergleich verwenden, wobei noch nicht ganz steht, wie der durchgeführt wird

In [ ]:
def show_move_list(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)
